In [ ]:
from collections import defaultdict
import pyspark.sql.types as stypes
import operator
import math
from pyspark.sql import Row
import pyspark.sql.functions as sfunc

In [ ]:
t = spark.read.json('gs://lbanor/dataproc_example/intermediary/2017-11-02',
                     schema=_load_users_matrix_schema())

In [ ]:
t.head(3)

In [ ]:
def load_users_matrix_schema():
    """Loads schema with data type [user, [(sku, score), (sku, score)]]
    :rtype: `pyspark.sql.type.StructType`
    :returns: schema speficiation for user -> (sku, score) data.
    """
    return stypes.StructType(fields=[
        stypes.StructField("user", stypes.StringType()),
         stypes.StructField('interactions', stypes.ArrayType(
          stypes.StructType(fields=[stypes.StructField('item', 
           stypes.StringType()), stypes.StructField('score', 
            stypes.FloatType())])))])

In [ ]:
stypes.ArrayType?

In [ ]:
schema = stypes.StructType(fields=[stypes.StructField('user', stypes.StringType()),
                                   stypes.StructField('interactions', stypes.ArrayType(
                                    stypes.StructType(fields=[stypes.StructField('item', stypes.StringType()),
                                                              stypes.StructField('score', stypes.FloatType())])))])
#schema = stypes.StructType(fields=[stypes.StructField('user', stypes.StringType())])

In [ ]:
sqlContext.createDataFrame?

In [ ]:
data = sc.parallelize([['user0', [('sku0', 0.5), ('sku1', 1.5)]]])
#data = sc.parallelize([['user0']])

In [ ]:
df = sqlContext.createDataFrame(data, schema=schema)

In [ ]:
df.head(3)

In [ ]:
t.createOrReplaceTempView('test1')

In [ ]:
spark.sql("select user, interactions.score from(select user, explode(interactions) interactions from test1)").createOrReplaceTempView('tt')

In [ ]:
spark.sql("select * from tt limit 10").head(10)

In [ ]:
t2.head(3)

In [ ]:
def process_intersections(row):
    r = []
    for i in range(len(row)):
        for j in range(i + 1, len(row)):
            r.append((row[i][0], row[j][0], row[i][1] * row[j][1]))
    return r

In [ ]:
def process_norms(row):
    return [(e[0], e[1] ** 2) for e in row]

In [ ]:
def process_intersections(row):
    return [('sku0', 'sku1', 1.), ('sku1', 'sku2', 0.4)]

In [ ]:
sqlContext.udf.register?

In [ ]:
sqlContext.udf.register("correlations", process_intersections, returnType=stypes.ArrayType(stypes.StructType(fields=[stypes.StructField('sku0', stypes.StringType()), stypes.StructField('sku1', stypes.StringType()), stypes.StructField('cor', stypes.FloatType())])))

In [ ]:
sqlContext.udf.register("SQUARED", process_norms, returnType=stypes.ArrayType(stypes.StructType(fields=[stypes.StructField('sku0', stypes.StringType()), stypes.StructField('norm', stypes.FloatType())])))

In [ ]:
query = """
SELECT
a.sku0 sku0,
a.sku1 sku1,
a.cor / (b.norm * c.norm) similarity
FROM(
    SELECT
      inter.sku0 sku0,
      inter.sku1 sku1,
      SUM(inter.cor) cor
    FROM(
      SELECT
        EXPLODE(CORRELATIONS(interactions)) inter
      FROM test1
      WHERE SIZE(interactions) BETWEEN 2 AND 20
      )
    GROUP BY 1, 2
    ) a
JOIN (
SELECT 
  sku0,
  norm
FROM test2
) b
ON a.sku0 = b.sku0
JOIN (
SELECT 
  sku0,
  norm
FROM test2
) c
ON a.sku1 = c.sku0
"""

In [ ]:
query2 = """
SELECT
  norms.sku0 sku0,
  SQRT(SUM(norms.norm)) norm
FROM(
  SELECT
    EXPLODE(SQUARED(interactions)) norms
  FROM test1
  WHERE SIZE(interactions) BETWEEN 2 AND 20
  )
GROUP BY 1
"""

In [ ]:
query3 = """
SELECT
  *
FROM test2
"""

In [ ]:
type(spark)

In [ ]:
spark.sql(query3).head(3)

In [ ]:
cors = spark.sql(query)
cors.createOrReplaceTempView('test3')

In [ ]:
t3 = spark.sql(query2)
t3.createOrReplaceTempView('test2')

In [ ]:
t3.head(2)

In [ ]:
sfunc.size?

In [ ]:
query_final ="""
SELECT
sku0 as item,
COLLECT_LIST(STRUCT(sku1 as item, similarity)) similarity_items
FROM(
SELECT
  *
FROM(
SELECT
  sku0,
  sku1,
  similarity
FROM test3
) UNION ALL
(
SELECT
  sku1 as sku0,
  sku0 as sku1,
  similarity
FROM test3
)
)
GROUP BY 1
"""

In [ ]:
final = spark.sql(query_final)

In [ ]:
final.head(3)

In [ ]:
final.write.json('gs://lbanor/dataproc_example/df_naive/', compression='gzip', mode='overwrite')

In [ ]:
final.head(2)

In [ ]:
sfunc.struct?

In [ ]:
def load_neighbor_schema(self):
    """Loads neighborhood schema for similarity matrix
    :rtype: `pyspark.sql.types.StructField`
    :returns: schema of type ["key", [("key", "value")]]
    """
    return stypes.StructType(fields=[
            stypes.StructField("item", stypes.StringType()),
             stypes.StructField("similarity_items", stypes.ArrayType(
              stypes.StructType(fields=[
               stypes.StructField("item", stypes.StringType()),
                stypes.StructField("similarity", stypes.FloatType())])))])

In [ ]:
empty = sqlContext.createDataFrame(sc.emptyRDD(), schema=load_users_matrix_schema())

In [ ]:
empty.union(t).head(3)

In [ ]:
type(sqlContext)

In [ ]:
type(spark)

In [ ]:
from pyspark.sql.context import SQLContext

In [ ]:
SQLContext?

In [ ]:
sess = SparkSession(sc)

In [ ]:
sess.udf.register

In [ ]:
sess.createDataFrame?

In [ ]:
final.count()

In [ ]:
int('5', 16) * pow(16, 3) + int('9', 16) * pow(16, 2)

In [ ]:
import json

In [ ]:
d = sc.parallelize([(0, u'{"colA":"Value1,Value4"}'), (52, u'{"colA":"Value2"}')])

In [ ]:
d.map(lambda x: ([json.loads(x[1])['colA']])).collect()